In [1]:
# Import libs
import os
import datetime
import tensorflow as tf
from tensorflow import keras

In [2]:
# Load dataset
# get data_dir for tranining
current_path = os.getcwd()

img_height = 224
img_width = 224
batch_size = 64

# set train dataset
train_data_dir = os.path.join(current_path, 'Vegetable_Images\\train')
train_ds = keras.utils.image_dataset_from_directory(
    train_data_dir,
    label_mode='categorical',
    image_size = (img_height, img_width),
    batch_size = batch_size
)

# set validation dataset
val_data_dir = os.path.join(current_path, 'Vegetable_Images\\validation')
val_ds = keras.utils.image_dataset_from_directory(
    val_data_dir,
    label_mode='categorical',
    image_size = (img_height, img_width),
    batch_size = batch_size
)

class_names = train_ds.class_names
class_num = len(class_names)

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.


In [3]:
# Build model
ResNet50_excludeTop = keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(img_height, img_width, 3)
)
ResNet50_excludeTop.trainable = False

Top_layers = keras.layers.GlobalAveragePooling2D()(ResNet50_excludeTop.output)
Top_layers = keras.layers.Dense(units=1024, activation='relu', use_bias=True)(Top_layers)
Top_layers = keras.layers.Dropout(rate=0.25)(Top_layers)
Top_layers = keras.layers.Dense(units=class_num, activation='softmax', use_bias=True)(Top_layers)

ResNet50_Model = keras.Model(inputs=ResNet50_excludeTop.input, outputs=Top_layers)

ResNet50_Model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

ResNet50_Model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 25,701,263 (98.04 MB)

 Trainable params: 2,113,551 (8.06 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [4]:
# Train model
logdir = os.path.join("logs", datetime.datetime.now().strftime("ResNet50_Model-%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

history = ResNet50_Model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tensorboard_callback]
)

Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 535s 2s/step - accuracy: 0.8125 - loss: 0.7026 - val_accuracy: 0.9923 - val_loss: 0.0309
Epoch 2/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 523s 2s/step - accuracy: 0.9954 - loss: 0.0296 - val_accuracy: 0.9967 - val_loss: 0.0140
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 522s 2s/step - accuracy: 0.9981 - loss: 0.0134 - val_accuracy: 0.9973 - val_loss: 0.0090
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 522s 2s/step - accuracy: 0.9992 - loss: 0.0075 - val_accuracy: 0.9980 - val_loss: 0.0072
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 522s 2s/step - accuracy: 0.9994 - loss: 0.0049 - val_accuracy: 0.9987 - val_loss: 0.0054
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 523s 2s/step - accuracy: 0.9999 - loss: 0.0032 - val_accuracy: 0.9993 - val_loss: 0.0041
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 522s 2s/step - accuracy: 0.9998 - loss: 0.0026 - val_accuracy: 0.9987 - val_loss: 0.0051
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 525s 2s/step - accuracy: 1.0000 - loss: 0.0020 - val_accu

In [5]:
# Save model as keras
ResNet50_Model.save('saved_models\ResNet50_Model.keras')